<h1><center>Laboratorio 6: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno: José Luis Cádiz



### **Link de repositorio de GitHub:** `https://github.com/JLCdata`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- Fecha de entrega: 12/11/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [175]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
#!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


<p align="center"><iframe width="560" height="315" src="https://www.youtube.com/embed/MW4d8_LLYp8?rel=0&amp;controls=0&amp;showinfo=0" frameborder="1" allowfullscreen></iframe></p>

# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la farbicación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [2]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail["InvoiceDate"]= pd.to_datetime(df_retail["InvoiceDate"], yearfirst=True, infer_datetime_format=True)
df_retail


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
400942,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
400943,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
400944,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
400945,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

# Creación de tabla auxiliar que contiene fechas de compra y total de compra de cada boleta de los clientes

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- **Recency (R)**: Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, $t_{m-i+1}; t_{m}$ es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Nota:** Para la $fechafinal$ utilice la fecha máxima del dataset más 1 día.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

# Desarrollo

In [165]:
# Generamos columna de compra total por producto
df_retail["Total"]=df_retail["Quantity"]*df_retail["Price"]

In [166]:
# Tabla auxiliar 1 que contiene el total de la compra de cada boleta
VentasTotalInvoice=df_retail.groupby(["Invoice"]).sum().reset_index(drop=False)[["Invoice","Total"]]
VentasTotalInvoice.head()

,Invoice,Total
0,489434,505.30
1,489435,145.80
2,489436,630.33
3,489437,310.75
4,489438,2286.24


In [168]:
# Tabla auxiliar 2

# Cruce de la tabla  master de retail (df_retail) con las ventas de cada invoice

# Outer Join
auxiliar=pd.merge(left=df_retail,
         right=VentasTotalInvoice,
         on="Invoice",
         how="outer",
         sort=True)
# Se eliminan los tags que no permiten diferenciar las filas repetidas
auxiliar.drop(["StockCode","Description","Quantity","Price","Country","Total_x"], 
axis=1,
inplace=True)

# Se eliminan filas repetidas, obteniendose el total del invoice para cada customer 
auxiliar.drop_duplicates(inplace=True,keep='first')
auxiliar.rename(columns={"Total_y":"Total"},inplace=True)
auxiliar.head()

,Invoice,InvoiceDate,Customer ID,Total
0,489434,2009-12-01 07:45:00,13085.0,505.30
8,489435,2009-12-01 07:46:00,13085.0,145.80
12,489436,2009-12-01 09:06:00,13078.0,630.33
31,489437,2009-12-01 09:08:00,15362.0,310.75
54,489438,2009-12-01 09:24:00,18102.0,2286.24


In [169]:
# Generacion de función auxiliar para Periodicity (Se usa con apply)
auxiliar2=auxiliar.copy()
def Perio(x):
    x=x["InvoiceDate"]
    xf=x
    xi=x.shift(1)
    deltas=xf-xi
    deltas.dropna(inplace=True)
    std=deltas.std()
    return std

**Respuesta:**

In [170]:
def custom_features(df_retail):
    
    # Fecha final del registro + 1 día
    t_fecha_final=max(df_retail["InvoiceDate"])+datetime.timedelta(days=1)
    
    # Generacion Length
    Length=pd.DataFrame((df_retail.groupby("Customer ID").max()["InvoiceDate"]-df_retail.groupby("Customer ID").min()["InvoiceDate"]).apply(lambda x: x.days)).rename(columns={"InvoiceDate":"Length"})
    
    # Generación de Recency
    auxiliar1=auxiliar.copy()
    auxiliar1["sum_diff"]=(t_fecha_final-auxiliar1["InvoiceDate"]).apply(lambda x: x.days)
    Recency=pd.DataFrame(auxiliar1.groupby("Customer ID").sum()["sum_diff"]/auxiliar1.groupby("Customer ID").size()).rename(columns={0:"Recency"})
    
    # Generacion Monetary
    # Se agrupa por cliente y se promedian sus compras hechas en diferentes fechas
    Monetary=auxiliar.groupby("Customer ID").mean().rename(columns={"Total":"Monetary"})
    
    # Generacion Frequency
    Frequency=pd.DataFrame(auxiliar.groupby("Customer ID").size(),columns=["Frequency"])
    
    # Generación de Periodicity
    Periodicity=pd.DataFrame(auxiliar2.groupby("Customer ID").apply(lambda x: Perio(x)),columns=["Periodicity"])
    Periodicity["Periodicity"]=Periodicity["Periodicity"].apply(lambda x: x.days)
    
    # Merges
    
    # Merges 1
    full=Length.merge(Recency,how="outer",left_index=True,right_index=True)

    # Merges 2
    full=full.merge(Monetary,how="outer",left_index=True,right_index=True)

    # Merges 3
    full=full.merge(Frequency,how="outer",left_index=True,right_index=True)

    # Merges 4
    full=full.merge(Periodicity,how="outer",left_index=True,right_index=True)
    
    # Drop nan
    full.dropna(inplace=True)
    
    return full

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [214]:
# Método estático (No ejecutar este)
class MinMax(BaseEstimator, TransformerMixin):
    
    @staticmethod
    def fit_transform(X):
        minmax_scaler = MinMaxScaler()
        Xnew = pd.DataFrame(minmax_scaler.fit_transform(X),columns=X.columns,index=X.index)
        return Xnew
    
#MinMax.fit_transform(features_clientes)

In [363]:
# Método no estático Custom min max -> Ejecutar version dataframe completo
class MinMax(BaseEstimator, TransformerMixin):
    def __init__(self,):
        self
    
    def fit(self,X):
        self.max=[]
        self.min=[]
        for i in X.columns:
            self.max.append(max(X[i]))
            self.min.append(min(X[i]))
        return self
    
    def transform(self,X):
        X = X.copy()
        for index,tag in enumerate(X.columns):
            X[tag]=(X[tag]-self.min[index])/(self.max[index]-self.min[index])
        
        return X
minmax=MinMax()

In [364]:
# Fit estandarización
minmax.fit(features_clientes)
# Estandarización
minmax.transform(features_clientes).head()

,Length,Recency,Monetary,Frequency,Periodicity
Customer ID,,,,,
12346.0,0.525469,0.878251,0.003086,0.039409,0.141176
12349.0,0.485255,0.410173,0.117618,0.000000,0.396078
12356.0,0.117962,0.065796,0.157252,0.000000,0.047059
12358.0,0.954424,0.494167,0.119752,0.000000,0.015686
12359.0,0.825737,0.667755,0.055683,0.014778,0.176471


### 1.3.2 `T-SNA` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo T-sne de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [356]:
# 1) Generación de features
Features = FunctionTransformer(custom_features)
features_clientes=Features.transform(df_retail)
features_clientes.head()

,Length,Recency,Monetary,Frequency,Periodicity
Customer ID,,,,,
12346.0,196,325.181818,33.896364,11,36.0
12349.0,181,158.000000,890.380000,3,101.0
12356.0,44,35.000000,1186.766667,3,12.0
12358.0,356,188.000000,906.336667,3,4.0
12359.0,308,250.000000,427.226667,6,45.0


In [388]:
# 2) y 3)

# Cols

cols_transform=features_clientes.columns

# Selección de features a estandarizar (todas)

preprocessing_transformer = ColumnTransformer(transformers=[('MinMax', MinMax(),  [i for i in cols_transform])])

# Creación del pipeline

pipeline = Pipeline([('Preprocessing', preprocessing_transformer),('TSNE', TSNE(n_components=2, random_state=42))])

In [389]:
# Ejecución del pipeline
features_tsne_embedded = pipeline.fit_transform(features_clientes)
features_tsne_embedded=pd.DataFrame(features_tsne_embedded,columns=['x_tsne','y_tsne'])
features_tsne_embedded.head()

,x_tsne,y_tsne
0,-28.951897,-15.270649
1,-5.339707,-21.602982
2,-14.751554,-72.218430
3,13.347960,55.555706
4,-8.663645,15.616383


In [390]:
# Visualización de baja dimensionalidad
fig = go.Figure()


fig.add_trace(go.Scatter(x=features_tsne_embedded['x_tsne'], y=features_tsne_embedded['y_tsne'],
                    mode='markers',
                    line_color='#D117CA'))

fig.update_layout(
    title="Visualización de baja dimensionalidad",
    xaxis_title='x_tsne',
    yaxis_title='y_tsne',
    font=dict(
        size=12,
        color="black"
    )
)      
fig.show()

In [396]:
# Visualización de datos estandarizados
fig = px.scatter_matrix(MinMax().fit_transform(features_clientes).sample(100), height=800)
fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero permutando el algoritmo de reducción de dimensionalidad por k-means.

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [424]:
# Iteración de inercias sobre cada número de clusters ejecutado
inertias = [
    [i, Pipeline([('Preprocessing', preprocessing_transformer),('k-means', KMeans(n_clusters=i, random_state=0))]).fit(features_clientes)[1].inertia_]
    for i in range(1, 21)]

# Se guardan inercias y número de clusters en un data frame
inertias = pd.DataFrame(inertias, columns=["n° clusters", "inertia"])

In [425]:
# Visualización del método del codo
px.line(inertias, x="n° clusters", y="inertia", title="Método del Codo con K-Means")

**A partir del gráfico anterior, se observa un quiebre en la caida de inertia entre 5 y 6 clusters, por lo que se eligen 5 clusters tratando de minimizar la cantidad de grupos, de modo de no dispersar demasiado la caracterización de clientes.**

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [435]:
kmeans = KMeans(n_clusters=5, random_state=0)
labels=kmeans.fit(features_tsne_embedded).labels_
features_tsne_embedded["Labels"]=labels
features_tsne_embedded["Labels"]=features_tsne_embedded["Labels"].astype("category")

**Calcular promedios, medianas, histogramas, por clusters etc**

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [437]:
# x and y given as DataFrame columns
import plotly.express as px
fig = px.scatter(features_tsne_embedded, x="x_tsne", y="y_tsne",color="Labels")

fig.update_layout(
    title="Visualización de segmentación de clientes de retail en baja dimensionalidad",
    xaxis_title='x_tsne',
    yaxis_title='y_tsne',
    font=dict(
        size=12,
        color="black"
    )
)  
fig.show()

**Escribir**

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab** solo para tareas. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>